In [1]:
import pandas as pd
from datetime import datetime, date

In [2]:
import time

In [2]:
import gspread

In [3]:
gc = gspread.service_account('..secret.json') #Create access to spreadsheet

In [4]:
spreadsheet_file = gc.open('Data Dashboard')

In [5]:
sheet = spreadsheet_file.worksheet('Dashboard')

In [15]:
pd.options.mode.chained_assignment = None 

# 0. Data upload and cleaning 

In [7]:
from sqlalchemy import create_engine
import pymysql

sqlEngine = create_engine()
dbConnection = sqlEngine.connect()
df = pd.read_sql("select * from users", dbConnection, parse_dates =['created_date'])

In [4]:
df.iloc[:, 2:].head()

,Acquisition Source / Medium,Acquisition Campaign,status,created_date,user_type,analytics_status,subs_created,subs_cancelled
0,(direct)/(none),(direct),ACTIVE,2022-12-14,0,FREE_TRIAL,NaN,NaN
1,(direct)/(none),(not set),ACTIVE,2022-12-14,0,FREE_TRIAL,NaN,NaN
2,google/organic,(not set),ACTIVE,2022-12-14,0,FREE_TRIAL,NaN,NaN
3,(direct)/(none),(not set),ACTIVE,2022-12-14,1,SECONDARY,NaN,NaN
4,(direct)/(none),(not set),PENDING_IMPORT,2022-12-14,0,PENDING_IMPORT,NaN,NaN


In [8]:
df = df[df['user_type'] == 0] #Only primary users 

In [9]:
def first_ten(string):
    if isinstance(string, str):
        sstring = string[:10]
        sstring= pd.to_datetime(sstring)
        return sstring

In [10]:
#reshape subs_created to the YMD format with datetime 
df['subs_created'] = df['subs_created'].apply(first_ten)
df['subs_cancelled'] = df['subs_cancelled'].apply(first_ten)

## 1. Data deduplication

In [23]:
nan = len(df[df['client_uuid'].isnull()]) #NaN in client ID 
nan_cid_df  = df[df['client_uuid'].isnull()]
nans_dupl_emails = len(nan_cid_df[nan_cid_df['frontend_email'].duplicated()])
print(f'There are {nan} users without recorded IDs.')

In [24]:
nnulls = df[df['client_uuid'].notnull()]
nnuls_dupl = len(nnulls[nnulls['client_uuid'].duplicated()][['client_uuid']])
df[df['client_uuid'].duplicated()][['client_uuid']].value_counts().head() #Counts by CID
max_dupl_cid = df[df['client_uuid'].duplicated()][['client_uuid']].value_counts().head(1).item()
dupl_cid = df[df['client_uuid'].duplicated()][['client_uuid']].nunique().item()
print(f'Among rows with recorded ID, there are {dupl_cid} duplicated client IDs across {nnuls_dupl} rows, some of them occuring as many as {max_dupl_cid} times in the period.')

In [25]:
#Dropping all rows with duplicated IDs, except for NaN values in cid 
data_deduplc = df[df['client_uuid'].isnull() | ~df[df['client_uuid'].notnull()].duplicated(subset='client_uuid',keep='first')]

In [27]:
print(f'After CID deduplication and data cleaning, there are {len(data_deduplc)} unique primary users in the period.')

# 2. Defining dates 

In [28]:
#Finding last month
import datetime

today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
last_month_year = int(last_month.strftime("%Y"))
last_month_month = int(last_month.strftime("%m"))

In [29]:
#Finding all days of the last month 
from calendar import monthrange

def get_datetime_range(year, month):
    nb_days = monthrange(year, month)[1]
    return [datetime.date(year, month, day) for day in range(1, nb_days+1)]

date_list = get_datetime_range(last_month_year, last_month_month)  

# 3. Last month overview

In [43]:
nond_reg = df[df['created_date'].isin(date_list)]
#Last months registrations including all duplicates 

In [46]:
dedupl_reg = data_deduplc[data_deduplc['created_date'].isin(date_list)]
#Removing all duplicate cIDs. 

In [48]:
#By unique emails 
sql_currmonth = nond_reg.groupby(['frontend_email']).count().sort_values('Acquisition Source / Medium', ascending = False)

In [51]:
#By analytics status
nond_reg.drop_duplicates(subset='frontend_email',keep='first')['analytics_status'].value_counts(dropna = False)

In [52]:
#By status
nond_reg.drop_duplicates(subset='frontend_email',keep='first')['status'].value_counts(dropna= False)

In [54]:
nond_paid  = nond_reg [nond_reg['analytics_status'] =='PAID'] 
paid_currmonth = len(nond_paid.groupby('frontend_email').count()) 

nond_cancelled = nond_reg [nond_reg['analytics_status'] =='CANCELLED']
cancelled_currmonth  = len(nond_cancelled.groupby('frontend_email').count())
 
nond_audit = nond_reg [nond_reg['status'] =='AUDIT']
audit_currmonth = len(nond_audit.groupby('frontend_email').count())

nond_pending = nond_reg [nond_reg['analytics_status'] =='PENDING_IMPORT']
pending_currmonth = len(nond_pending.groupby('frontend_email').count())

nond_expired = nond_reg [nond_reg['analytics_status'] =='TRIAL_EXPIRED']
expired_currmonth = len(nond_expired.groupby('frontend_email').count())

vreg = len(sql_currmonth) - audit_currmonth

print(f'Total registrations: {len(sql_currmonth)}')
print(f'Total audit: {audit_currmonth}')
print(f'Total registrations excluding audit: {vreg}')
print(f'Total pending registrations: {pending_currmonth}')
print(f'Total paid users from the current months cohort: {paid_currmonth}')
print(f'Total cancelled users from the current months cohort: {cancelled_currmonth}')
print(f'Total expired free trial: {expired_currmonth}')

# 4. Updating KPI dashboard in Google Sheets

In [57]:
#Get list of months 
months_list_str = pd.date_range('2015-05-01',last_month, freq='MS').strftime("%Y%m").tolist()

In [58]:
months_list = pd.date_range('2015-06-01',today, freq='MS').tolist()

In [59]:
index = ['all', 'audit', 'valid_reg', 'pending', 'paid', 'cancelled', 'active']
result = {}

In [61]:
#Updating data in cohorts and updating Google Sheets file 
for month in months_list:
    last_month = month - datetime.timedelta(days=1)
    last_month_year = int(last_month.strftime("%Y"))
    last_month_month = int(last_month.strftime("%m"))
    date_list = get_datetime_range(last_month_year, last_month_month)
    
    #Fetching all numbers grouped by email
    nond_reg = df[df['created_date'].isin(date_list)]
    sql_currmonth = len(nond_reg.groupby(['frontend_email']).count().sort_values('Acquisition Source / Medium', ascending = False))
    nond_paid  = nond_reg [nond_reg['analytics_status'] =='PAID'] 
    paid_currmonth = len(nond_paid.groupby('frontend_email').count()) 
    nond_cancelled = nond_reg [nond_reg['analytics_status'] =='CANCELLED']
    cancelled_currmonth  = len(nond_cancelled.groupby('frontend_email').count())
    nond_audit = nond_reg [nond_reg['status'] =='AUDIT']
    audit_currmonth = len(nond_audit.groupby('frontend_email').count())
    nond_pending = nond_reg [nond_reg['analytics_status'] =='PENDING_IMPORT']
    pending_currmonth = len(nond_pending.groupby('frontend_email').count())
    nond_expired = nond_reg [nond_reg['analytics_status'] =='TRIAL_EXPIRED']
    expired_currmonth = len(nond_expired.groupby('frontend_email').count())
    vreg = sql_currmonth - audit_currmonth
    
    #Combining the results  
    result[last_month] = [sql_currmonth,audit_currmonth, vreg, pending_currmonth, paid_currmonth, cancelled_currmonth, paid_currmonth+cancelled_currmonth]
    resulting_file = pd.DataFrame(result, index = index)  
    last_month_m = datetime.datetime.strftime(last_month, "%B %Y")
    
    #Writing to the file
    
    ## 1. Define col nr; if none is found return last_col + 1 
    find_col_nr = sheet.find(last_month_m)
    if find_col_nr == None:
        col_nr = len(sheet.row_values(1))+1
        sheet.update_cell(1, col_nr, last_month_m)
    else:
        col_nr = find_col_nr.col
    
    ##2. Write to 4 cells using the col nr and the row nr: audit, valid_reg, pending, paid, cancelled 
    sheet.update_cell(9, col_nr, vreg)
    time.sleep(2) #Sleep used because of API quota limitations 
    sheet.update_cell(10, col_nr, pending_currmonth)
    time.sleep(2)
    sheet.update_cell(20, col_nr, paid_currmonth)
    time.sleep(2)
    sheet.update_cell(21, col_nr, cancelled_currmonth)
    time.sleep(5)